In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.4 MB/s eta 0:00:00


## BERT

In [2]:
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [3]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla V100-SXM2-16GB


In [4]:
# 네이버 영화 리뷰 데이터 다운로드

!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 21.47 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Updating files: 100% (14737/14737), done.


In [5]:
# 코랩 가상 드라이브 상 nsmc 드라이브 생성된 path로 불러오기
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


In [6]:
document_bert = ["[CLS] " + str(s) + " [SEP]" for s in train.document]
document_bert[:5]

['[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]',
 '[CLS] 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [SEP]',
 '[CLS] 너무재밓었다그래서보는것을추천한다 [SEP]',
 '[CLS] 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [SEP]',
 '[CLS] 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [SEP]']

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in document_bert]
print(tokenized_texts[0])

['[CLS]', '아', '더', '##빙', '.', '.', '진', '##짜', '짜', '##증', '##나', '##네', '##요', '목', '##소', '##리', '[SEP]']


In [8]:
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
input_ids[0]

array([   101,   9519,   9074, 119005,    119,    119,   9708, 119235,
         9715, 119230,  16439,  77884,  48549,   9284,  22333,  12692,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [9]:
# 위대로 하면 학습속도 느림! 실제 데이터와 padding 있는 곳 인덱스 지정

attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [10]:
train_inputs, validation_inputs, train_labels, validation_labels = \
train_test_split(input_ids, train['label'].values, random_state=42, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks,
                                                       input_ids,
                                                       random_state=42,
                                                       test_size=0.1)

In [11]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [12]:
BATCH_SIZE = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

In [13]:
sentences = test['document']
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = test['label'].values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [14]:
# hugging face!

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [15]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

epochs = 4

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [17]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        loss = outputs[0]

        total_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

        model.zero_grad()


    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")


    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:01:46.
  Batch 1,000  of  4,219.    Elapsed: 0:03:31.
  Batch 1,500  of  4,219.    Elapsed: 0:05:17.
  Batch 2,000  of  4,219.    Elapsed: 0:07:02.
  Batch 2,500  of  4,219.    Elapsed: 0:08:47.
  Batch 3,000  of  4,219.    Elapsed: 0:10:33.
  Batch 3,500  of  4,219.    Elapsed: 0:12:18.
  Batch 4,000  of  4,219.    Elapsed: 0:14:03.

  Average training loss: 0.39
  Training epcoh took: 0:14:49

Running Validation...
  Accuracy: 0.85
  Validation took: 0:00:30

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:01:45.
  Batch 1,000  of  4,219.    Elapsed: 0:03:31.
  Batch 1,500  of  4,219.    Elapsed: 0:05:16.
  Batch 2,000  of  4,219.    Elapsed: 0:07:02.
  Batch 2,500  of  4,219.    Elapsed: 0:08:47.
  Batch 3,000  of  4,219.    Elapsed: 0:10:33.
  Batch 3,500  of  4,219.    Elapsed: 0:12:18.
  Batch 4,000  of  4,219.    Elapsed: 0:14:03.

  Average training loss: 0.29
  

## BART

In [ ]:
from transformers import BartConfig, BartTokenizer, BartModel

config = BartConfig.from_pretrained("facebook/bart-large")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")
model = BartModel.from_pretrained("facebook/bart-large")

In [ ]:
# Trainer arguments
lr = 3e-5
stop = 3
epoch = 100
batch = 4
seed = 42
device = 'cuda'

### initializing

In [ ]:
class AwsS3Downloader(object):
    def __init__(
        self,
        aws_access_key_id=None,
        aws_secret_access_key=None,
    ):
        self.resource = boto3.Session(
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
        ).resource("s3")
        self.client = boto3.client(
            "s3",
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            config=Config(signature_version=UNSIGNED),
        )

    def __split_url(self, url: str):
        if url.startswith("s3://"):
            url = url.replace("s3://", "")
        bucket, key = url.split("/", maxsplit=1)
        return bucket, key

    def download(self, url: str, local_dir: str):
        bucket, key = self.__split_url(url)
        filename = os.path.basename(key)
        file_path = os.path.join(local_dir, filename)

        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        meta_data = self.client.head_object(Bucket=bucket, Key=key)
        total_length = int(meta_data.get("ContentLength", 0))

        downloaded = 0

        def progress(chunk):
            nonlocal downloaded
            downloaded += chunk
            done = int(50 * downloaded / total_length)
            sys.stdout.write(
                "\r{}[{}{}]".format(file_path, "█" * done, "." * (50 - done))
            )
            sys.stdout.flush()

        try:
            with open(file_path, "wb") as f:
                self.client.download_fileobj(bucket, key, f, Callback=progress)
            sys.stdout.write("\n")
            sys.stdout.flush()
        except:
            raise Exception(f"downloading file is failed. {url}")
        return file_path

def download(url, chksum=None, cachedir=".cache"):
    cachedir_full = os.path.join(os.getcwd(), cachedir)
    os.makedirs(cachedir_full, exist_ok=True)
    filename = os.path.basename(url)
    file_path = os.path.join(cachedir_full, filename)
    if os.path.isfile(file_path):
        if hashlib.md5(open(file_path, "rb").read()).hexdigest()[:10] == chksum:
            print(f"using cached model. {file_path}")
            return file_path, True

    s3 = AwsS3Downloader()
    file_path = s3.download(url, cachedir_full)
    if chksum:
        assert (
            chksum == hashlib.md5(open(file_path, "rb").read()).hexdigest()[:10]
        ), "corrupted file!"
    return file_path, False

def get_kobart_tokenizer(cachedir=".cache"):
    """Get KoGPT2 Tokenizer file path after downloading"""
    tokenizer = {
        "url": "s3://skt-lsl-nlp-model/KoBART/tokenizers/kobart_base_tokenizer_cased_cf74400bce.zip",
        "chksum": "cf74400bce",
    }
    file_path, is_cached = download(
        tokenizer["url"], tokenizer["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.expanduser(cachedir)
    if (
        not os.path.exists(os.path.join(cachedir_full, "emji_tokenizer"))
        or not is_cached
    ):
        if not is_cached:
            shutil.rmtree(
                os.path.join(cachedir_full, "emji_tokenizer"), ignore_errors=True
            )
        zipf = ZipFile(os.path.expanduser(file_path))
        zipf.extractall(path=cachedir_full)
    tok_path = os.path.join(cachedir_full, "emji_tokenizer/model.json")
    tokenizer_obj = PreTrainedTokenizerFast(
        tokenizer_file=tok_path,
        bos_token="<s>",
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        mask_token="<mask>",
    )
    return tokenizer_obj

def get_pytorch_kobart_model(ctx="cpu", cachedir=".cache"):
    pytorch_kobart = {
        "url": "s3://skt-lsl-nlp-model/KoBART/models/kobart_base_cased_ff4bda5738.zip",
        "chksum": "ff4bda5738",
    }
    model_zip, is_cached = download(
        pytorch_kobart["url"], pytorch_kobart["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.join(os.getcwd(), cachedir)
    model_path = os.path.join(cachedir_full, "kobart_from_pretrained")
    if not os.path.exists(model_path) or not is_cached:
        if not is_cached:
            shutil.rmtree(model_path, ignore_errors=True)
        zipf = ZipFile(os.path.expanduser(model_zip))
        zipf.extractall(path=cachedir_full)
    return model_path

In [ ]:
def make_df(phase):
    work_dir = "/MyDrive/Colab Notebooks"
    if phase == 'train':
        tmp = work_dir+'/kubig_NLP/1.Training/라벨링데이터/TL1'
    else:
        tmp = work_dir+'/kubig_NLP/2.Validation/라벨링데이터/VL1'
    listdir = os.listdir(tmp)
    df = pd.DataFrame({}, columns = ['genre', 'text', 'label'])
    for i in listdir:
        files = os.listdir(f'{tmp}/{i}/2~3sent')
        for f in tqdm(files):
            with open(f'{tmp}/{i}/2~3sent/{f}', 'r', encoding='utf-8') as json_file:
                j = json.loads(json_file.read())
                df2 = pd.DataFrame.from_dict([{'genre' : i,
                                               'text'  : j['Meta(Refine)']['passage'],
                                               'label' : j['Annotation']['summary1']}])
                df = pd.concat([df, df2])
    return df

In [ ]:
train = pd.read_csv('train.tsv', sep='\t')
train.columns = ['text', 'label']
train

,text,label
0,일제 강제징용피해자들이 지난 2월 발의된 ‘근로정신대 피해자 지원법률안’의 조속한 ...,13일 광주시의회 시민 소통실에서 시민단체 '근로정신대 할머니와 함께 하는 시민모임...
1,문재인 대통령이 이르면 9일 중폭 개각을 단행할 것으로 보인다. 여권 관계자는 8일...,"문재인 대통령이 이르면 9일 장관 3~4명, 장관급 3명을 교체하는 개각을 단행하고..."
2,전남 어촌마을 정주여건 개선 ‘탄력’ 해수부 공모서 고흥 득량도·강진 신전 등 4개...,"해양수산부 어촌종합개발 거점 개발사업에 전남 고흥 득량도, 강진 신전, 완도 충동,..."
3,여자친구 알몸을 무단으로 촬영해 인터넷 성인 카페에 유포한 남성이 재판에 넘겨졌다....,피해자 A씨는 가해자인 B씨의 엄벌을 촉구하며 국민청원에 피해사실을 공개하였고 결국...
4,대구시의회 배지숙 의장이 개회를 선언하고 있다. 대구시의회(의장 배지숙)는 14일부...,대구시의회가 14일 제1차 본회의를 시작으로 제265회 임시회를 개최하고 ‘대구시시...
...,...,...
34237,"민생 구하기부터 나서야” 자유한국당은 7일 더불어민주당을 향해 ‘재판 불복, 민심 ...",김병준 비상대책위원장은 민주당 이해찬 대표가 대선불복은 절대 받아들일 수 없는 것이...
34238,"민주평화통일자문회의 당진시협의회(회장 김광일, 이하 민주평통) 한성희 수석부회장이 ...","지난 20일, 한성희 수석부회장이 국민훈장을 수상했고, 앞서 지난 19일에는 박병원..."
34239,충남도서관 전경 충남도서관이 한국문화공간건축학회로부터 도서관 부문 '2018 한국문...,"올해로 4회째를 맞는 한국문화공간상에서 충남 도서관이 공공성과 접근성, 사용자들의 ..."
34240,전남도의회는 10일 열린 제 335회 임시회 본회의에서 이광일(더불어민주당·여수 1...,전남도의회는 헌재가 현행 해상경계선을 인정하고 현행 해상경계선을 경계선으로 확정할 ...


### 전처리

In [ ]:
train = train[:-2]
val = train[-2:].reset_index(drop=True)

In [ ]:
def preprocss(df):
    df.text = df.text.apply(lambda x : re.sub('\n', ' ',  x))
    df.text = df.text.apply(lambda x : re.sub('[^ .,가-힣A-Za-z0-9]', ' ',  x))
    df.text = df.text.apply(lambda x : re.sub(' +', ' ',  x).strip())
    return df

train = preprocss(train)
val = preprocss(val)

C:\Users\hist\AppData\Local\Temp\ipykernel_11616\4244231608.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.text = df.text.apply(lambda x : re.sub('\n', ' ',  x))
C:\Users\hist\AppData\Local\Temp\ipykernel_11616\4244231608.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.text = df.text.apply(lambda x : re.sub('[^ .,가-힣A-Za-z0-9]', ' ',  x))
C:\Users\hist\AppData\Local\Temp\ipykernel_11616\4244231608.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [ ]:
val.loc[0, 'text']

'민주평화통일자문회의 당진시협의회 회장 김광일, 이하 민주평통 한성희 수석부회장이 국민훈장 석류장 을 수상했다. 이와 함께 박병원 자문위원이 민주평통 의장 대통령 표창을, 전철수 자문위원이 충남도지사표창 수상하며 영예를 안았다. 지난 20일 백범김구기념관 컨벤션홀에서 국민훈장 수여식이 진행된 가운데 한성희 수석부회장이 국민훈장을 수상했다. 앞서 지난 19일에는 의장표창 및 도지사표창 전수식이 순천향대학교 유니토피아관에서 열려 박병원 전철수 자문위원이 민주평통 의장표창과 충남도지사표창을 수상했다. 유공자문위원 표창 수여식은 민주평통의 국내 외 자문위원 2만여 명을 대상으로 하는 평화통일 기반 구축과 통일준비 활동에 공적이 있는 자문위원에 대한 포상으로, 올해 충남지역 위원 777명 에서 국민훈장 1명, 의장표창 17명이 수상했다. 한성희 수석부회장 오 한성희 수석부회장 국민훈장 충남에서 유일하게 국민훈장을 수상한 한성희 수석부회장은 사할린 동포와 교류확대 등 국제외교활동으로 남북한의 이질감을 해소하고 국제사회의 우호증진과 신뢰를 구축하는데 기여했다는 평을 받으며 이번 상을 수상했다. 한성희 수석부회장은 15기부터 민주평통 자문위원으로 활동했으며 16기 간사를 거쳐 현재 18기에서 수석부회장을 맡아 활동하고 있다. 한성희 수석부회장은 한반도에 새로운 평화의 시대가 시작된 올해 국민훈장 석류장 을 받게 돼 무한한 영광으로 생각한다 며 앞으로 더욱 잘하라는 채찍질이라 생각할 것 이라고 말했다. 덧붙여 통일로 가는 과정에 다양한 의견이 있지만 우리 소원은 하나 라며 한반도의 번영을 위해 문재인 대통령의 9월 평양공동선언 의 합의사항을 차근히 이행해 나갈 때 우리는 평화와 번영의 터전을 후손에게 물려줄 수 있다고 생각한다 고 말했다. 더불어 2019년은 3.1운동이 100주년 되는 해 라며 선조들이 남겨 준 자유롭고 독립된 터전을 잘 보존해 아름답고 살기 좋은 나라를 만드는데 노력할 것 이라고 말했다. 박병원 자문위원 오 박병원 자문위원 대통령표창 민주평통 의장인 대통령

In [ ]:
val.loc[0, 'label']

'지난 20일, 한성희 수석부회장이 국민훈장을 수상했고, 앞서 지난 19일에는 박병원·전철수 자문위원이 민주평통 의장표창과 충남도지사표창을 수상하는등, 올해 충남지역에서 국민훈장 1명, 의장표창 17명이 수상했다.'

In [ ]:
train.text.str.len().min()

401

### Dataset

In [ ]:
class KoBARTSumDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, ignore_index=-100):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.df = df
        self.len = len(self.df)

        self.pad_index = self.tokenizer.pad_token_id
        self.ignore_index = ignore_index

    def add_padding_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.pad_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs

    def add_ignored_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.ignore_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs

    def __getitem__(self, idx):
        instance = self.df.iloc[idx]
        input_ids = self.tokenizer.encode(instance['text'])
        input_ids = np.append(input_ids, self.tokenizer.eos_token_id)
        input_ids = self.add_padding_data(input_ids)
        input_ids = np.insert(input_ids, 0 , self.tokenizer.bos_token_id)

        label_ids = self.tokenizer.encode(instance['label'])
        label_ids.append(self.tokenizer.eos_token_id)
        label_ids.insert(0, self.tokenizer.bos_token_id)

        dec_input_ids = [self.tokenizer.eos_token_id]
        dec_input_ids += label_ids[:-1]
        dec_input_ids = self.add_padding_data(dec_input_ids)
        label_ids = self.add_ignored_data(label_ids)

        input_ids = torch.tensor(np.array(input_ids)).long()
        decoder_input_ids = torch.tensor(np.array(dec_input_ids)).long()

        return {'input_ids': input_ids,
                'attention_mask': input_ids.ne(self.tokenizer.pad_token_id).float(),
                'decoder_input_ids': decoder_input_ids,
                'decoder_attention_mask' : decoder_input_ids.ne(self.tokenizer.pad_token_id).float(),
                'labels': np.array(label_ids, dtype=np.int_)}

        # return {'input_ids': np.array(input_ids, dtype=np.int_),
        #         'decoder_input_ids': np.array(dec_input_ids, dtype=np.int_),
        #         'labels': np.array(label_ids, dtype=np.int_)}

    def __len__(self):
        return self.len
train_dataset = KoBARTSumDataset(train, PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1'), 512)
val_dataset = KoBARTSumDataset(val, PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1'), 512)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
train_dataset[-1]

{'input_ids': tensor([    0, 17484, 23051, 25003, 17484, 28941, 16122, 14188, 14912, 18381,
         18408, 25507, 15163, 21744, 16259, 26011, 14188, 14912, 18381, 11224,
         15309, 16135, 14049, 14126, 28699, 14105, 18332, 26150, 26352, 24486,
         19378, 14188, 14912, 18381, 15978, 14441, 14043, 16316, 14563, 21519,
         15056, 17308,  9908, 14454, 14685, 15637, 14059, 14442, 14500, 24466,
         16022, 14633, 17072, 14280, 19085, 12124, 14585, 25013, 21119, 16652,
         25700, 13590, 14700, 16398, 12024, 14685,  9115, 17359, 14152, 27731,
         18074, 16671, 10675, 14282, 17834, 19503, 14191, 14185, 14188, 14912,
         18381, 18408, 13587, 15519, 19923, 15615, 14322, 14685, 18381, 15978,
         28977, 17810, 18549, 21744,   243, 15379, 15063, 15748, 14912, 18381,
           243, 16011, 17676,   243, 14998, 11224, 14048, 19444, 15130, 10338,
         17975, 15224, 17432, 17484, 28941, 16154, 15641, 15355, 16405, 16564,
         14442, 15335, 14653, 17465, 14

### Model

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v1')
        self.model.train()
        self.bos_token = '<s>'
        self.eos_token = '</s>'

        self.tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')
        self.pad_token_id = self.tokenizer.pad_token_id

    def forward(self, inputs):
        # attention_mask = inputs['input_ids'].ne(self.pad_token_id).float()
        # decoder_attention_mask = inputs['decoder_input_ids'].ne(self.pad_token_id).float()

        # return self.model(input_ids=inputs['input_ids'],
        #                   attention_mask=attention_mask,
        #                   decoder_input_ids=inputs['decoder_input_ids'],
        #                   decoder_attention_mask=decoder_attention_mask,
        #                   labels=inputs['labels'], return_dict=True)
        return self.model(input_ids=inputs['input_ids'],
                  attention_mask=inputs['attention_mask'],
                  decoder_input_ids=inputs['decoder_input_ids'],
                  decoder_attention_mask=inputs['decoder_attention_mask'],
                  labels=inputs['labels'], return_dict=True)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('gogamza/kobart-base-v1')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=tokenizer.pad_token_id)

In [ ]:
rouge = Rouge()
def compute_metrics(pred):
    preds, labels = pred

    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    print('Text\n', val.text[0])
    print(f'Summarize\nGold : {labels[0]}\nGen : {preds[0]}')

    labels = ['\n'.join(labels)]
    preds = ['\n'.join(preds)]
    score = rouge.get_scores(preds, labels, avg=True)
    return {
        "ROUGE-1" : score['rouge-1'],
        "ROUGE-2" : score['rouge-2'],
        "ROUGE-L" : score['rouge-l'],
    }

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=1, eta_min=0, last_epoch=-1)

lr_scheduler = transformers.get_cosine_schedule_with_warmup(optimizer=optimizer,
                                                            num_warmup_steps=100,
                                                            num_training_steps=epoch * len(train_dataset) * batch,
                                                            last_epoch = -1)

In [ ]:
args = Seq2SeqTrainingArguments(run_name = f'KoBARTSum',
                                output_dir= f"models",
                                evaluation_strategy="steps",
                                eval_steps=100,
                                save_steps=100,
                                logging_steps=100,
                                save_total_limit = 2,

                                per_device_train_batch_size=batch,
                                per_device_eval_batch_size=batch,
                                gradient_accumulation_steps=16,
                                num_train_epochs=epoch,

                                load_best_model_at_end=True,
                                fp16=True,
                                do_train=True,
                                do_eval=True,
                                predict_with_generate=True,)

trainer = Seq2SeqTrainer(model=model,
                         tokenizer=tokenizer,
                         args=args,

                         train_dataset=train_dataset,
                         eval_dataset=val_dataset,

                         compute_metrics=compute_metrics,
                         optimizers=(optimizer, lr_scheduler),
                         # callbacks=[EarlyStoppingCallback(early_stopping_patience=stop)],
                         data_collator=collator,)
trainer.train()

Using cuda_amp half precision backend
***** Running training *****
  Num examples = 73338
  Num Epochs = 100
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 114500
  Number of trainable parameters = 123859968
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
100,3.291500,1.540322,"{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}"
200,1.564300,1.314538,"{'r': 0.47058823529411764, 'p': 0.42105263157894735, 'f': 0.44444443945987655}","{'r': 0.375, 'p': 0.3333333333333333, 'f': 0.35294117148788934}","{'r': 0.47058823529411764, 'p': 0.42105263157894735, 'f': 0.44444443945987655}"
300,1.492100,1.235838,"{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}"
400,1.445100,1.189054,"{'r': 0.6470588235294118, 'p': 0.6470588235294118, 'f': 0.6470588185294118}","{'r': 0.4375, 'p': 0.4375, 'f': 0.43749999500000003}","{'r': 0.6470588235294118, 'p': 0.6470588235294118, 'f': 0.6470588185294118}"
500,1.418900,1.216791,"{'r': 0.47058823529411764, 'p': 0.4444444444444444, 'f': 0.4571428521469388}","{'r': 0.375, 'p': 0.35294117647058826, 'f': 0.363636358640955}","{'r': 0.47058823529411764, 'p': 0.4444444444444444, 'f': 0.4571428521469388}"
600,1.401000,1.263261,"{'r': 0.4117647058823529, 'p': 0.3888888888888889, 'f': 0.3999999950040816}","{'r': 0.3125, 'p': 0.29411764705882354, 'f': 0.30303029803489445}","{'r': 0.4117647058823529, 'p': 0.3888888888888889, 'f': 0.3999999950040816}"
700,1.377100,1.202939,"{'r': 0.6470588235294118, 'p': 0.6470588235294118, 'f': 0.6470588185294118}","{'r': 0.4375, 'p': 0.4375, 'f': 0.43749999500000003}","{'r': 0.6470588235294118, 'p': 0.6470588235294118, 'f': 0.6470588185294118}"
800,1.362000,1.157833,"{'r': 0.5294117647058824, 'p': 0.47368421052631576, 'f': 0.4999999950154322}","{'r': 0.375, 'p': 0.3333333333333333, 'f': 0.35294117148788934}","{'r': 0.5294117647058824, 'p': 0.47368421052631576, 'f': 0.4999999950154322}"
900,1.359600,1.220365,"{'r': 0.23529411764705882, 'p': 0.2222222222222222, 'f': 0.2285714235755103}","{'r': 0.1875, 'p': 0.17647058823529413, 'f': 0.18181817682277332}","{'r': 0.23529411764705882, 'p': 0.2222222222222222, 'f': 0.2285714235755103}"
1000,1.343600,1.148808,"{'r': 0.6470588235294118, 'p': 0.6470588235294118, 'f': 0.6470588185294118}","{'r': 0.4375, 'p': 0.4375, 'f': 0.43749999500000003}","{'r': 0.6470588235294118, 'p': 0.6470588235294118, 'f': 0.6470588185294118}"


***** Running Evaluation *****
  Num examples = 2
  Batch size = 4
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}



Text
 다큐 온 은 이 시대가 요구하는 모든 온 이야기를, 갓 지은 밥처럼 따뜻하고 영양이 풍부하게 만들어 온 , 시청자들이 필요로 하는 바로 그 시점에 ON 전달한다. 탄소중립시대, 우리가 나무를 심는 이유. 인간은 단순히 숨 쉬는 것만으로도 이산화탄소를 배출하고 있다. 과연 그 양은 얼마나 될까 실험을 통해 살펴본 결과 001시간 기준 약 27g. 인간이 80세까지 배출하는 양은 약 20여 톤. 이 탄소를 상쇄하려면 소나무 160그루가 필요하다. 그러나 인간이 일상생활을 영위하는 과정에는 무수히 많은 탄소가 배출된다. 의식주를 포함한 모든 생활 속에서 전기를 사용해야 하는데, 이 에너지의 대부분은 화석 연료에 의한 발전으로 가능하기 때문이다. 기본적인 생활 중에 배출되는 탄소발자국을 따라가 본다.
Summarize
Gold : 인간은 80세까지 20여 톤의 이산화탄소를 배출하고 있다.
Gen : 다큐 온은 시청자들이 필요로 하는 모든 온 이야기를, 갓 지은 밥처럼 따뜻하고


Saving model checkpoint to models\checkpoint-100
Configuration saved in models\checkpoint-100\config.json
Configuration saved in models\checkpoint-100\generation_config.json
Model weights saved in models\checkpoint-100\pytorch_model.bin
tokenizer config file saved in models\checkpoint-100\tokenizer_config.json
Special tokens file saved in models\checkpoint-100\special_tokens_map.json
Deleting older checkpoint [models\checkpoint-1000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2
  Batch size = 4
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}



...

Saving model checkpoint to models\checkpoint-48500
Configuration saved in models\checkpoint-48500\config.json
Configuration saved in models\checkpoint-48500\generation_config.json
Model weights saved in models\checkpoint-48500\pytorch_model.bin
tokenizer config file saved in models\checkpoint-48500\tokenizer_config.json
Special tokens file saved in models\checkpoint-48500\special_tokens_map.json
Deleting older checkpoint [models\checkpoint-48400] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2
  Batch size = 4
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}



Text
 다큐 온 은 이 시대가 요구하는 모든 온 이야기를, 갓 지은 밥처럼 따뜻하고 영양이 풍부하게 만들어 온 , 시청자들이 필요로 하는 바로 그 시점에 ON 전달한다. 탄소중립시대, 우리가 나무를 심는 이유. 인간은 단순히 숨 쉬는 것만으로도 이산화탄소를 배출하고 있다. 과연 그 양은 얼마나 될까 실험을 통해 살펴본 결과 001시간 기준 약 27g. 인간이 80세까지 배출하는 양은 약 20여 톤. 이 탄소를 상쇄하려면 소나무 160그루가 필요하다. 그러나 인간이 일상생활을 영위하는 과정에는 무수히 많은 탄소가 배출된다. 의식주를 포함한 모든 생활 속에서 전기를 사용해야 하는데, 이 에너지의 대부분은 화석 연료에 의한 발전으로 가능하기 때문이다. 기본적인 생활 중에 배출되는 탄소발자국을 따라가 본다.
Summarize
Gold : 인간은 80세까지 20여 톤의 이산화탄소를 배출하고 있다.
Gen : 인간은 80세까지 20여 톤의 이산화탄소를 배출하고 있다.


Saving model checkpoint to models\checkpoint-48600
Configuration saved in models\checkpoint-48600\config.json
Configuration saved in models\checkpoint-48600\generation_config.json
Model weights saved in models\checkpoint-48600\pytorch_model.bin
tokenizer config file saved in models\checkpoint-48600\tokenizer_config.json
Special tokens file saved in models\checkpoint-48600\special_tokens_map.json
Deleting older checkpoint [models\checkpoint-48500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2
  Batch size = 4
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}



Text
 다큐 온 은 이 시대가 요구하는 모든 온 이야기를, 갓 지은 밥처럼 따뜻하고 영양이 풍부하게 만들어 온 , 시청자들이 필요로 하는 바로 그 시점에 ON 전달한다. 탄소중립시대, 우리가 나무를 심는 이유. 인간은 단순히 숨 쉬는 것만으로도 이산화탄소를 배출하고 있다. 과연 그 양은 얼마나 될까 실험을 통해 살펴본 결과 001시간 기준 약 27g. 인간이 80세까지 배출하는 양은 약 20여 톤. 이 탄소를 상쇄하려면 소나무 160그루가 필요하다. 그러나 인간이 일상생활을 영위하는 과정에는 무수히 많은 탄소가 배출된다. 의식주를 포함한 모든 생활 속에서 전기를 사용해야 하는데, 이 에너지의 대부분은 화석 연료에 의한 발전으로 가능하기 때문이다. 기본적인 생활 중에 배출되는 탄소발자국을 따라가 본다.
Summarize
Gold : 인간은 80세까지 20여 톤의 이산화탄소를 배출하고 있다.
Gen : 인간은 80세까지 20여 톤의 이산화탄소를 배출하고 있다.


Saving model checkpoint to models\checkpoint-48700
Configuration saved in models\checkpoint-48700\config.json
Configuration saved in models\checkpoint-48700\generation_config.json
Model weights saved in models\checkpoint-48700\pytorch_model.bin
tokenizer config file saved in models\checkpoint-48700\tokenizer_config.json
Special tokens file saved in models\checkpoint-48700\special_tokens_map.json
Deleting older checkpoint [models\checkpoint-48600] due to args.save_total_limit


### Inference

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('models/checkpoint-100600').to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
out = model.generate(input_ids = torch.tensor(val_dataset[0]['input_ids'])[None,:].to(device))
tokenizer.decode(out.squeeze().tolist(), skip_special_tokens=True)

C:\Users\hist\AppData\Local\Temp\ipykernel_11616\1642432242.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = model.generate(input_ids = torch.tensor(val_dataset[0]['input_ids'])[None,:].to(device))
C:\Users\hist\miniconda3\envs\py38\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'민주평화통일자문회의 당진시협의회 한 수석부회장과 박 자문위원'

In [ ]:
tokenizer.decode(val_dataset[0]['input_ids'])

'<s> 민주평화통일자문회의 당진시협의회 회장 김광일, 이하 민주평통 한성희 수석부회장이 국민훈장 석류장 을 수상했다. 이와 함께 박병원 자문위원이 민주평통 의장 대통령 표창을, 전철수 자문위원이 충남도지사표창 수상하며 영예를 안았다. 지난 20일 백범김구기념관 컨벤션홀에서 국민훈장 수여식이 진행된 가운데 한성희 수석부회장이 국민훈장을 수상했다. 앞서 지난 19일에는 의장표창 및 도지사표창 전수식이 순천향대학교 유니토피아관에서 열려 박병원 전철수 자문위원이 민주평통 의장표창과 충남도지사표창을 수상했다. 유공자문위원 표창 수여식은 민주평통의 국내 외 자문위원 2만여 명을 대상으로 하는 평화통일 기반 구축과 통일준비 활동에 공적이 있는 자문위원에 대한 포상으로, 올해 충남지역 위원 777명 에서 국민훈장 1명, 의장표창 17명이 수상했다. 한성희 수석부회장 오 한성희 수석부회장 국민훈장 충남에서 유일하게 국민훈장을 수상한 한성희 수석부회장은 사할린 동포와 교류확대 등 국제외교활동으로 남북한의 이질감을 해소하고 국제사회의 우호증진과 신뢰를 구축하는데 기여했다는 평을 받으며 이번 상을 수상했다. 한성희 수석부회장은 15기부터 민주평통 자문위원으로 활동했으며 16기 간사를 거쳐 현재 18기에서 수석부회장을 맡아 활동하고 있다. 한성희 수석부회장은 한반도에 새로운 평화의 시대가 시작된 올해 국민훈장 석류장 을 받게 돼 무한한 영광으로 생각한다 며 앞으로 더욱 잘하라는 채찍질이라 생각할 것 이라고 말했다. 덧붙여 통일로 가는 과정에 다양한 의견이 있지만 우리 소원은 하나 라며 한반도의 번영을 위해 문재인 대통령의 9월 평양공동선언 의 합의사항을 차근히 이행해 나갈 때 우리는 평화와 번영의 터전을 후손에게 물려줄 수 있다고 생각한다 고 말했다. 더불어 2019년은 3.1운동이 100주년 되는 해 라며 선조들이 남겨 준 자유롭고 독립된 터전을 잘 보존해 아름답고 살기 좋은 나라를 만드는데 노력할 것 이라고 말했다. 박병원 자문위원 오 박병원 자문위원 대통령표창 민주평통 의장인

In [ ]:
val.label[0]

'지난 20일, 한성희 수석부회장이 국민훈장을 수상했고, 앞서 지난 19일에는 박병원·전철수 자문위원이 민주평통 의장표창과 충남도지사표창을 수상하는등, 올해 충남지역에서 국민훈장 1명, 의장표창 17명이 수상했다.'

In [ ]:
def infer(text):
    if type(text) == int:
        out = model.generate(input_ids = torch.tensor(train_dataset[text]['input_ids'])[None,:].to(device)).logits.argmax(2)
        result = tokenizer.decode(out[0])
    else:
        tmp = [tokenizer.bos_token_id] + tokenizer.encode(text) + [tokenizer.eos_token_id]
        out = model.generate(input_ids = torch.tensor(tmp)[None,:].to(device))
        result = tokenizer.decode(out[0])
    return print(result)

In [ ]:
text = '1일 오후 9시까지 최소 20만3220명이 코로나19에 신규 확진됐다. 또다시 동시간대 최다 기록으로, 사상 처음 20만명대에 진입했다. 방역 당국과 서울시 등 각 지방자치단체에 따르면 이날 0시부터 오후 9시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다. 국내 신규 확진자 수가 20만명대를 넘어선 것은 이번이 처음이다. 동시간대 최다 기록은 지난 23일 오후 9시 기준 16만1389명이었는데, 이를 무려 4만1831명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명 많다. 확진자 폭증은 3시간 전인 오후 6시 집계에서도 예견됐다. 오후 6시까지 최소 17만8603명이 신규 확진돼 동시간대 최다 기록(24일 13만8419명)을 갈아치운 데 이어 이미 직전 0시 기준 역대 최다 기록도 넘어섰다. 역대 최다 기록은 지난 23일 0시 기준 17만1451명이었다. 17개 지자체별로 보면 서울 4만6938명, 경기 6만7322명, 인천 1만985명 등 수도권이 12만5245명으로 전체의 61.6%를 차지했다. 서울과 경기는 모두 동시간대 기준 최다로, 처음으로 각각 4만명과 6만명을 넘어섰다. 비수도권에서는 7만7975명(38.3%)이 발생했다. 제주를 제외한 나머지 지역에서 모두 동시간대 최다를 새로 썼다. 부산 1만890명, 경남 9909명, 대구 6900명, 경북 6977명, 충남 5900명, 대전 5292명, 전북 5150명, 울산 5141명, 광주 5130명, 전남 4996명, 강원 4932명, 충북 3845명, 제주 1513명, 세종 1400명이다. 집계를 마감하는 자정까지 시간이 남아있는 만큼 2일 0시 기준으로 발표될 신규 확진자 수는 이보다 더 늘어날 수 있다. 이에 따라 최종 집계되는 확진자 수는 21만명 안팎을 기록할 수 있을 전망이다. 한편 전날 하루 선별진료소에서 이뤄진 검사는 70만8763건으로 검사 양성률은 40.5%다. 양성률이 40%를 넘은 것은 이번이 처음이다. 확산세가 계속 거세질 수 있다는 얘기다. 이날 0시 기준 신규 확진자는 13만8993명이었다. 이틀 연속 13만명대를 이어갔다.'
infer(text)

</s><s> 코로나19 사상 처음 20만 이상 인구가 신규 확진되었다.</s>


In [ ]:
text

'1일 오후 9시까지 최소 20만3220명이 코로나19에 신규 확진됐다. 또다시 동시간대 최다 기록으로, 사상 처음 20만명대에 진입했다. 방역 당국과 서울시 등 각 지방자치단체에 따르면 이날 0시부터 오후 9시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다. 국내 신규 확진자 수가 20만명대를 넘어선 것은 이번이 처음이다. 동시간대 최다 기록은 지난 23일 오후 9시 기준 16만1389명이었는데, 이를 무려 4만1831명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명 많다. 확진자 폭증은 3시간 전인 오후 6시 집계에서도 예견됐다. 오후 6시까지 최소 17만8603명이 신규 확진돼 동시간대 최다 기록(24일 13만8419명)을 갈아치운 데 이어 이미 직전 0시 기준 역대 최다 기록도 넘어섰다. 역대 최다 기록은 지난 23일 0시 기준 17만1451명이었다. 17개 지자체별로 보면 서울 4만6938명, 경기 6만7322명, 인천 1만985명 등 수도권이 12만5245명으로 전체의 61.6%를 차지했다. 서울과 경기는 모두 동시간대 기준 최다로, 처음으로 각각 4만명과 6만명을 넘어섰다. 비수도권에서는 7만7975명(38.3%)이 발생했다. 제주를 제외한 나머지 지역에서 모두 동시간대 최다를 새로 썼다. 부산 1만890명, 경남 9909명, 대구 6900명, 경북 6977명, 충남 5900명, 대전 5292명, 전북 5150명, 울산 5141명, 광주 5130명, 전남 4996명, 강원 4932명, 충북 3845명, 제주 1513명, 세종 1400명이다. 집계를 마감하는 자정까지 시간이 남아있는 만큼 2일 0시 기준으로 발표될 신규 확진자 수는 이보다 더 늘어날 수 있다. 이에 따라 최종 집계되는 확진자 수는 21만명 안팎을 기록할 수 있을 전망이다. 한편 전날 하루 선별진료소에서 이뤄진 검사는 70만8763건으로 검사 양성률은 40.5%다. 양성률이 40%를 넘은 것은 이번이 처음이다. 확산세가 계속 거세질 수 있다는 얘기다